## Importación de LightGlue

In [1]:
## En teoria ya esta en el repo, pero por si acaso

from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .

fatal: destination path 'LightGlue' already exists and is not an empty directory.
/Users/manu/Proyecto Final/Proyecto-Final-Gondolieri/LightGlue


/Users/manu/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Modelo prueba

In [5]:
import torch
import cv2

from LightGlue.lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from LightGlue.lightglue.utils import load_image, rbd
from LightGlue.lightglue import viz2d
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
import time

#### Defino parametros de configuración

In [15]:
# Defino parámetros de configuración
# Ajustamos los hiperparámetros con los nuevos valores
n_layers = 5  # Mantener en 7
flash = True
mp = True
# Ajustar los hiperparámetros a valores más intermedios
depth_confidence = 1  # Reducir para encontrar más coincidencias
width_confidence = 1 # Aumentar ligeramente para evitar perder puntos clave
filter_threshold = 0.3  # Volver a un umbral más bajo para evitar filtrar demasiados puntos

# Definir parámetros adicionales
min_matches = 10  # Número mínimo de correspondencias necesarias para considerar que el objeto está presente
max_keypoints = None  # Número máximo de puntos clave a extraer

# Configurar el dispositivo y los modelos
device = torch.device("cpu")
# Ajustar el extractor de puntos clave (SuperPoint) y matcher (LightGlue)
extractor = SuperPoint(max_num_keypoints=max_keypoints).eval().to(device)

# Configurar el dispositivo y los modelos
matcher = LightGlue(
    features="superpoint",
    n_layers=n_layers,  
    flash=flash,  
    mp=mp,  
    depth_confidence=depth_confidence,  # Ajustar para permitir más procesamiento
    width_confidence=width_confidence,  # Menos poda de puntos clave
    filter_threshold=filter_threshold  # Umbral más bajo para más coincidencias
).eval().to(device)

def calculate_iou_with_threshold(m_kpts1, x_min, x_max, y_min, y_max, gondola_split, iou_threshold=0.5):
    # Contar los puntos de coincidencia dentro de la bounding box (intersección)
    in_bbox_count = sum(
        1 for x, y in m_kpts1 if x_min <= x <= x_max and y_min <= y <= y_max
    )
    
    # Contar todos los puntos de coincidencia dentro de la imagen (unión)
    total_points_in_image = sum(
        1 for x, y in m_kpts1 if 0 <= x <= gondola_split.shape[2] and 0 <= y <= gondola_split.shape[1]
    )
    
    # Evitar la división por cero
    if total_points_in_image == 0:
        return 0, False  # Retorna 0 de IoU y False para indicar que no es una buena coincidencia
    
    # Calcular IoU como la proporción de puntos dentro de la bounding box respecto al total de puntos
    iou = in_bbox_count / total_points_in_image
    
    # Verificar si el IoU cumple con el umbral definido
    is_good_match = iou >= iou_threshold
    
    return iou, is_good_match


In [22]:
# Cargar la imagen de la góndola
TP, FP, TN, FN = 0, 0, 0, 0

# Leer el archivo CSV de promociones
promociones_df = pd.read_csv('promociones.csv')
                

for gondola_image_path in Path("/Users/manu/Downloads/Mayo-Junio-Julio (teoricamente) 2").glob('*.jpeg'):
    gondola_split = load_image(gondola_image_path)
    print(f"Analizando la imagen: {gondola_image_path.name}")
    # Obtener dimensiones de la imagen
    height, width = gondola_split.shape[1:3]

    # Obtener tipo de gondola y marcas
    json_path = Path(gondola_image_path).stem + '.json'
    if not Path(json_path).exists():
        print(f"Archivo JSON no encontrado para la imagen: {gondola_image_path.name}")
        continue
    with open(json_path, 'r') as f:
        gondola_data = json.load(f)

    tipo = gondola_data["tipo"]
    marcas = gondola_data["marcas"]

    images_to_show = []
    # Calcular las posiciones de recorte
    top_half_end = int(height * 0.6)  # 60% de la altura
    bottom_half_start = int(height * 0.4)  # 40% de la altura (20% de overlap)
    overlap = top_half_end - bottom_half_start  # Definir el solapamiento

    # Crear las mitades
    top_half = gondola_split[:, :top_half_end, :]
    bottom_half = gondola_split[:, bottom_half_start:, :]
    half_size = top_half_end  # Definir half_size como la altura de la mitad superior

    # Ruta a la carpeta de productos
    for marca in marcas:
        product_folder = Path(f"/Users/manu/Downloads/Imagenes Productos 2/{tipo}/{marca}")

        for product_image_path in product_folder.glob('*'):  
            if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']: 

                # Obtener la descripción y el EAN nuevo del producto
                product_ean = product_image_path.stem.split(' - ')[-1]
                
                # Filtrar el DataFrame para encontrar coincidencias
                promociones_match = promociones_df[
                    (promociones_df['EAN NUEVO'] == product_ean) &
                    (promociones_df['Mes'] != 'Septiembre')
                ]
                
                # Si no hay coincidencias, continuar con el siguiente producto
                if promociones_match.empty:
                    print(f"No se encontró coincidencia en promociones para el producto: {product_image_path.name}")
                    continue
                print(f"Analizando el producto: {product_image_path.name}")

                product_image = load_image(product_image_path)
                
                # Inicializar contadores para las mitades
                top_half_matches = 0
                bottom_half_matches = 0
                # Convertir las imágenes a blanco y negro
                product_image = product_image.mean(dim=0, keepdim=True)
                top_half = top_half.mean(dim=0, keepdim=True)
                bottom_half = bottom_half.mean(dim=0, keepdim=True)
                # Procesar la mitad superior
                feats0_top = extractor.extract(product_image.to(device))
                feats1_top = extractor.extract(top_half.to(device))
                matches01_top = matcher({"image0": feats0_top, "image1": feats1_top})
                feats0_top, feats1_top, matches01_top = [rbd(x) for x in [feats0_top, feats1_top, matches01_top]]

                kpts0_top, kpts1_top, matches_top = feats0_top["keypoints"], feats1_top["keypoints"], matches01_top["matches"]
                top_half_matches = len(matches_top)

                # Procesar la mitad inferior
                feats0_bottom = extractor.extract(product_image.to(device))
                feats1_bottom = extractor.extract(bottom_half.to(device))
                matches01_bottom = matcher({"image0": feats0_bottom, "image1": feats1_bottom})
                feats0_bottom, feats1_bottom, matches01_bottom = [rbd(x) for x in [feats0_bottom, feats1_bottom, matches01_bottom]]

                kpts0_bottom, kpts1_bottom, matches_bottom = feats0_bottom["keypoints"], feats1_bottom["keypoints"], matches01_bottom["matches"]
                bottom_half_matches = len(matches_bottom)
            
                # Ajustar coordenadas para la mitad inferior
                adjusted_kpts1_bottom = kpts1_bottom.detach().clone()  
                adjusted_kpts1_bottom[..., 1] += bottom_half_start  
                print(f"Coincidencias en la mitad superior: {top_half_matches}, coincidencias en la mitad inferior: {bottom_half_matches}")
                
                # Elegir la mitad con más coincidencias
                if top_half_matches >= bottom_half_matches:
                    matches = matches_top
                    m_kpts1 = kpts1_top[matches[..., 1]]
                else:
                    matches = matches_bottom
                    m_kpts1 = adjusted_kpts1_bottom[matches[..., 1]]  # Usar las coordenadas ajustadas

                if len(matches) >= 15:
                    print(f"Objeto detectado con {len(matches)} correspondencias en la partición para el producto: {product_image_path.name}")

                    # Cargar las bounding boxes del archivo JSON
                    gondola_image_path = Path(gondola_image_path)
                    json_path = gondola_image_path.stem + '.json'
                    with open(json_path, 'r') as f:
                        bounding_boxes_data = json.load(f)

                    # Verificar si el producto tiene una bounding box en el JSON
                    product_name = product_image_path.stem
                    bounding_box_found = False
                    total_in_bbox_count = 0  # Contador total de puntos dentro de cualquier bounding box
                    bboxes = []  # Lista para almacenar las bounding boxes

                    for region in bounding_boxes_data["regions"]:
                        region_name = region["region_attributes"]["name"]
                        
                        # Verificar si el nombre del producto coincide con el de la región
                        if product_name in region_name:
                            # Obtener las coordenadas de la bounding box
                            bbox = region["shape_attributes"]
                            x_min = bbox["x"]
                            y_min = bbox["y"]
                            x_max = x_min + bbox["width"]
                            y_max = y_min + bbox["height"]
                            bounding_box_found = True
                            bboxes.append((x_min, y_min, x_max, y_max))  # Guardar bounding box

                            iou, is_good_match = calculate_iou_with_threshold(m_kpts1, x_min, x_max, y_min, y_max, gondola_split, iou_threshold=0.3)

                            if is_good_match:
                                print(f"La bounding box es una buena coincidencia con un IoU de {iou:.2f}")
                            else:
                                print(f"La bounding box no cumple con el umbral, IoU = {iou:.2f}")

                    # Visualización de puntos y bounding boxes con OpenCV
                    gondola_split_np = gondola_split.cpu().permute(1, 2, 0).numpy()  # Convertir a numpy
                    gondola_split_np = (gondola_split_np * 255).astype(np.uint8)  # Normalizar a [0, 255]
                    
                    # # Dibujar bounding boxes
                    # for bbox in bboxes:
                    #     if (bbox[2] <= gondola_split_np.shape[1] and bbox[0] >= 0 and
                    #         bbox[3] <= gondola_split_np.shape[0] and bbox[1] >= 0):
                    #         cv2.rectangle(gondola_split_np, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)  # Rojo

                    # # Dibujar puntos encontrados
                    # for x, y in m_kpts1:
                    #     cv2.circle(gondola_split_np, (int(x), int(y)), 5, (0, 255, 0), -1)  # Verde

                    # # Mostrar la imagen
                    # cv2.imshow(f'Puntos y Bounding Boxes para {product_name}', gondola_split_np)
                    # cv2.waitKey(20)  # Esperar hasta que se presione una tecla
                    # cv2.destroyAllWindows()


                    # Guardar la imagen en una lista para mostrar al final
                    images_to_show.append(gondola_split_np)

                    # Si se encontraron bounding boxes
                    if bounding_box_found:
                        # Si la mayoría de los puntos están dentro de las bounding boxes
                        if is_good_match:
                            print(f"Producto {product_name} detectado correctamente en la partición  con {total_in_bbox_count} puntos dentro de las bounding boxes.")
                            TP += 1  # Verdadero positivo
                        else:
                            print(f"Falso positivo para el producto {product_name} en la partición . No se encontraron puntos dentro de las bounding boxes.")
                            FP += 1  # Falso positivo
                    else:
                        print(f"Falso positivo: No se encontró bounding box para el producto {product_name}.")
                        FP += 1

                else:  # No hay suficientes coincidencias
                    # Cargar las bounding boxes del archivo JSON
                    gondola_image_path = Path(gondola_image_path)
                    json_path = gondola_image_path.stem + '.json'
                    with open(json_path, 'r') as f:
                        bounding_boxes_data = json.load(f)

                    # Verificar si el producto tiene una bounding box en el JSON
                    product_name = product_image_path.stem
                    bounding_box_found = any(product_name in region["region_attributes"]["name"] for region in bounding_boxes_data["regions"])

                    if bounding_box_found:
                        print(f"Falso negativo: Había una bounding box para el producto {product_name}, pero no se encontraron suficientes puntos de coincidencia.")
                        FN += 1  # Falso negativo
                    else:
                        print(f"Verdadero negativo: No se encontró ni bounding box ni suficientes coincidencias para el producto {product_name}.")
                        TN += 1  # Verdadero negativo


# Resultados finales
print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

    # Matriz de confusión
print("Matriz de confusión:")
print(f"              Predicted")
print(f"           | Pos | Neg ")
print(f"  Actual   |-----|-----|")
print(f"   Pos     | {TP}  | {FN} ")
print(f"   Neg     | {FP}  | {TN} ")

    # Calcular precisión, recall, F1-score y exactitud (accuracy)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1_score:.2f}")
print(f"Exactitud (Accuracy): {accuracy:.2f}")


Analizando la imagen: 28-MJJ-.jpeg
No se encontró coincidencia en promociones para el producto: REXONA M DEO AP AER HOM 12X89G_150ML - 7791293049502.jpg
No se encontró coincidencia en promociones para el producto: REXONA M DEO AP AER FUTB F 12X89G_150ML - 7791293049472.jpg
Analizando el producto: REXONA W CLIN AER EXTDR 96H 12X91G_150ML - 7891150064300.webp


/Users/manu/opt/anaconda3/lib/python3.9/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Coincidencias en la mitad superior: 5, coincidencias en la mitad inferior: 2
Verdadero negativo: No se encontró ni bounding box ni suficientes coincidencias para el producto REXONA W CLIN AER EXTDR 96H 12X91G_150ML - 7891150064300.
No se encontró coincidencia en promociones para el producto: REXONA M DEO AP AER V8 12X89G_150ML - 7791293049489.jpg
No se encontró coincidencia en promociones para el producto: REXONA M DEO ROL AP V8 12X50ML - 78944800.png
No se encontró coincidencia en promociones para el producto: REXONA FM DEO STI AP V8 12X50G - 75076788.jpeg
Analizando el producto: REXONA M CL AER IN FR 96H 12DX91G_150ML - 7891150089983.jpg
Coincidencias en la mitad superior: 4, coincidencias en la mitad inferior: 2
Falso negativo: Había una bounding box para el producto REXONA M CL AER IN FR 96H 12DX91G_150ML - 7891150089983, pero no se encontraron suficientes puntos de coincidencia.
No se encontró coincidencia en promociones para el producto: REXONA FW DEO SOFT SOLID CLASSIC12X48G - 7

libpng warning: iCCP: known incorrect sRGB profile


Coincidencias en la mitad superior: 1, coincidencias en la mitad inferior: 18
Objeto detectado con 18 correspondencias en la partición para el producto: PANTENE SH COLAGENO 400MLX12IT - 7500435137928.png
Falso positivo: No se encontró bounding box para el producto PANTENE SH COLAGENO 400MLX12IT - 7500435137928.
Analizando el producto: PNT SH PROVMIR DETOX 400MLX12IT - 7500435229821.png
Coincidencias en la mitad superior: 14, coincidencias en la mitad inferior: 41
Objeto detectado con 41 correspondencias en la partición para el producto: PNT SH PROVMIR DETOX 400MLX12IT - 7500435229821.png
La bounding box no cumple con el umbral, IoU = 0.00
La bounding box no cumple con el umbral, IoU = 0.00
Falso positivo para el producto PNT SH PROVMIR DETOX 400MLX12IT - 7500435229821 en la partición . No se encontraron puntos dentro de las bounding boxes.
Analizando el producto: PNT PROVMIR SH VOLUME 400MLX12IT - 7500435235297.png
Coincidencias en la mitad superior: 28, coincidencias en la mitad infer

In [4]:
# Cargar la imagen de la góndola
gondola_image_path = "/Users/manu/Downloads/Mayo-Junio-Julio (teoricamente) 2/7-MJJ-.jpeg"
gondola_split = load_image(gondola_image_path)
# Obtener dimensiones de la imagen
height, width = gondola_split.shape[1:3]

images_to_show = []
# Calcular las posiciones de recorte
top_half_end = int(height * 0.6)  # 60% de la altura
bottom_half_start = int(height * 0.4)  # 40% de la altura (20% de overlap)
overlap = top_half_end - bottom_half_start  # Definir el solapamiento

# Crear las mitades
top_half = gondola_split[:, :top_half_end, :]
bottom_half = gondola_split[:, bottom_half_start:, :]
half_size = top_half_end  # Definir half_size como la altura de la mitad superior

# Ruta a la carpeta de productos
product_folder = Path("/Users/manu/Downloads/Evagina")

TP, FP, TN, FN = 0, 0, 0, 0
for product_image_path in product_folder.glob('*'):  
    if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']: 
        print(f"Analizando el producto: {product_image_path.name}")

        product_image = load_image(product_image_path)
        
        # Inicializar contadores para las mitades
        top_half_matches = 0
        bottom_half_matches = 0
        # Convertir las imágenes a blanco y negro
        product_image = product_image.mean(dim=0, keepdim=True)
        top_half = top_half.mean(dim=0, keepdim=True)
        bottom_half = bottom_half.mean(dim=0, keepdim=True)
        # Procesar la mitad superior
        feats0_top = extractor.extract(product_image.to(device))
        feats1_top = extractor.extract(top_half.to(device))
        matches01_top = matcher({"image0": feats0_top, "image1": feats1_top})
        feats0_top, feats1_top, matches01_top = [rbd(x) for x in [feats0_top, feats1_top, matches01_top]]

        kpts0_top, kpts1_top, matches_top = feats0_top["keypoints"], feats1_top["keypoints"], matches01_top["matches"]
        top_half_matches = len(matches_top)

        # Procesar la mitad inferior
        feats0_bottom = extractor.extract(product_image.to(device))
        feats1_bottom = extractor.extract(bottom_half.to(device))
        matches01_bottom = matcher({"image0": feats0_bottom, "image1": feats1_bottom})
        feats0_bottom, feats1_bottom, matches01_bottom = [rbd(x) for x in [feats0_bottom, feats1_bottom, matches01_bottom]]

        kpts0_bottom, kpts1_bottom, matches_bottom = feats0_bottom["keypoints"], feats1_bottom["keypoints"], matches01_bottom["matches"]
        bottom_half_matches = len(matches_bottom)
       
        # Ajustar coordenadas para la mitad inferior
        adjusted_kpts1_bottom = kpts1_bottom.detach().clone()  
        adjusted_kpts1_bottom[..., 1] += bottom_half_start  
        print(f"Coincidencias en la mitad superior: {top_half_matches}, coincidencias en la mitad inferior: {bottom_half_matches}")
        
        # Elegir la mitad con más coincidencias
        if top_half_matches >= bottom_half_matches:
            matches = matches_top
            m_kpts1 = kpts1_top[matches[..., 1]]
        else:
            matches = matches_bottom
            m_kpts1 = adjusted_kpts1_bottom[matches[..., 1]]  # Usar las coordenadas ajustadas

        if len(matches) >= 8:
            print(f"Objeto detectado con {len(matches)} correspondencias en la partición para el producto: {product_image_path.name}")

            # Cargar las bounding boxes del archivo JSON
            gondola_image_path = Path(gondola_image_path)
            json_path = gondola_image_path.stem + '.json'
            with open(json_path, 'r') as f:
                bounding_boxes_data = json.load(f)

            # Verificar si el producto tiene una bounding box en el JSON
            product_name = product_image_path.stem
            bounding_box_found = False
            total_in_bbox_count = 0  # Contador total de puntos dentro de cualquier bounding box
            bboxes = []  # Lista para almacenar las bounding boxes

            for region in bounding_boxes_data["regions"]:
                region_name = region["region_attributes"]["name"]
                
                # Verificar si el nombre del producto coincide con el de la región
                if product_name in region_name:
                    # Obtener las coordenadas de la bounding box
                    bbox = region["shape_attributes"]
                    x_min = bbox["x"]
                    y_min = bbox["y"]
                    x_max = x_min + bbox["width"]
                    y_max = y_min + bbox["height"]
                    bounding_box_found = True
                    bboxes.append((x_min, y_min, x_max, y_max))  # Guardar bounding box

                    # Verificar si la bounding box cae dentro de la partición
                    if (x_max >= 0 and x_min <= gondola_split.shape[2] and
                        y_max >= 0 and y_min <= gondola_split.shape[1]):
                        
                        # Contar cuántos puntos de coincidencia están dentro de la bounding box
                        total_in_bbox_count += sum(
                            1 for x, y in m_kpts1 if x_min <= x <= x_max and y_min <= y <= y_max
                        )

            # Visualización de puntos y bounding boxes con OpenCV
            gondola_split_np = gondola_split.cpu().permute(1, 2, 0).numpy()  # Convertir a numpy
            gondola_split_np = (gondola_split_np * 255).astype(np.uint8)  # Normalizar a [0, 255]
            
            # Dibujar bounding boxes
            for bbox in bboxes:
                if (bbox[2] <= gondola_split_np.shape[1] and bbox[0] >= 0 and
                    bbox[3] <= gondola_split_np.shape[0] and bbox[1] >= 0):
                    cv2.rectangle(gondola_split_np, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)  # Rojo

            # Dibujar puntos encontrados
            for x, y in m_kpts1:
                cv2.circle(gondola_split_np, (int(x), int(y)), 5, (0, 255, 0), -1)  # Verde

            # Mostrar la imagen
            cv2.imshow(f'Puntos y Bounding Boxes para {product_name}', gondola_split_np)
            cv2.waitKey(20)  # Esperar hasta que se presione una tecla
            cv2.destroyAllWindows()


            # Guardar la imagen en una lista para mostrar al final
            images_to_show.append(gondola_split_np)

            # Si se encontraron bounding boxes
            if bounding_box_found:
                # Si la mayoría de los puntos están dentro de las bounding boxes
                if total_in_bbox_count > 0:
                    print(f"Producto {product_name} detectado correctamente en la partición  con {total_in_bbox_count} puntos dentro de las bounding boxes.")
                    TP += 1  # Verdadero positivo
                else:
                    print(f"Falso positivo para el producto {product_name} en la partición . No se encontraron puntos dentro de las bounding boxes.")
                    FP += 1  # Falso positivo
            else:
                print(f"Falso positivo: No se encontró bounding box para el producto {product_name}.")
                FP += 1

        else:  # No hay suficientes coincidencias
            # Cargar las bounding boxes del archivo JSON
            gondola_image_path = Path(gondola_image_path)
            json_path = gondola_image_path.stem + '.json'
            with open(json_path, 'r') as f:
                bounding_boxes_data = json.load(f)

            # Verificar si el producto tiene una bounding box en el JSON
            product_name = product_image_path.stem
            bounding_box_found = any(product_name in region["region_attributes"]["name"] for region in bounding_boxes_data["regions"])

            if bounding_box_found:
                print(f"Falso negativo: Había una bounding box para el producto {product_name}, pero no se encontraron suficientes puntos de coincidencia.")
                FN += 1  # Falso negativo
            else:
                print(f"Verdadero negativo: No se encontró ni bounding box ni suficientes coincidencias para el producto {product_name}.")
                TN += 1  # Verdadero negativo

    # Resultados finales
print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

        # Matriz de confusión
print("Matriz de confusión:")
print(f"              Predicted")
print(f"           | Pos | Neg ")
print(f"  Actual   |-----|-----|")
print(f"   Pos     | {TP}  | {FN} ")
print(f"   Neg     | {FP}  | {TN} ")

# Mostrar todas las imágenes al final
for idx, img in enumerate(images_to_show):
    cv2.imshow(f'Imagen {idx + 1}', img)
    cv2.waitKey(1)  # Esperar hasta que se presione una tecla para cada imagen
cv2.destroyAllWindows()

NameError: name 'load_image' is not defined

#### Carteles Promocionales

In [18]:
import os

# Crear una carpeta para guardar los recortes
output_folder = Path("/Users/manu/Downloads/detecciones")
output_folder.mkdir(parents=True, exist_ok=True)

for product_image_path in product_folder.glob('*'):  
    if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']: 
        print(f"Analizando el producto: {product_image_path.name}")

        product_image = load_image(product_image_path)

        # Convertir las imágenes a blanco y negro
        product_image = product_image.mean(dim=0, keepdim=True)
        top_half = top_half.mean(dim=0, keepdim=True)
        bottom_half = bottom_half.mean(dim=0, keepdim=True)

        # Procesar la mitad superior
        feats0_top = extractor.extract(product_image.to(device))
        feats1_top = extractor.extract(top_half.to(device))
        matches01_top = matcher({"image0": feats0_top, "image1": feats1_top})
        feats0_top, feats1_top, matches01_top = [rbd(x) for x in [feats0_top, feats1_top, matches01_top]]
        kpts0_top, kpts1_top, matches_top = feats0_top["keypoints"], feats1_top["keypoints"], matches01_top["matches"]
        top_half_matches = len(matches_top)

        # Procesar la mitad inferior
        feats0_bottom = extractor.extract(product_image.to(device))
        feats1_bottom = extractor.extract(bottom_half.to(device))
        matches01_bottom = matcher({"image0": feats0_bottom, "image1": feats1_bottom})
        feats0_bottom, feats1_bottom, matches01_bottom = [rbd(x) for x in [feats0_bottom, feats1_bottom, matches01_bottom]]
        kpts0_bottom, kpts1_bottom, matches_bottom = feats0_bottom["keypoints"], feats1_bottom["keypoints"], matches01_bottom["matches"]
        bottom_half_matches = len(matches_bottom)

        # Ajustar coordenadas para la mitad inferior
        adjusted_kpts1_bottom = kpts1_bottom.detach().clone()  
        adjusted_kpts1_bottom[..., 1] += bottom_half_start  
        print(f"Coincidencias en la mitad superior: {top_half_matches}, coincidencias en la mitad inferior: {bottom_half_matches}")

        # Elegir la mitad con más coincidencias
        if top_half_matches >= bottom_half_matches:
            matches = matches_top
            m_kpts1 = kpts1_top[matches[..., 1]]
        else:
            matches = matches_bottom
            m_kpts1 = adjusted_kpts1_bottom[matches[..., 1]]  # Usar las coordenadas ajustadas

        if len(matches) >= 8:
            print(f"Objeto detectado con {len(matches)} correspondencias en la partición para el producto: {product_image_path.name}")

            # Encontrar los límites de los puntos coincidentes
            x_coords = m_kpts1[..., 0].cpu().numpy()
            y_coords = m_kpts1[..., 1].cpu().numpy()
            x_min, x_max = max(0, x_coords.min()), min(gondola_split.shape[2], x_coords.max())
            y_min, y_max = max(0, y_coords.min()), min(gondola_split.shape[1], y_coords.max())

            # Agregar un pequeño margen para que el recorte no sea muy justo
            margin = 10
            x_min, x_max = int(max(0, x_min - margin)), int(min(gondola_split.shape[2], x_max + margin))
            y_min, y_max = int(max(0, y_min - margin)), int(min(gondola_split.shape[1], y_max + margin))

            # Recortar la imagen original en base a los puntos de coincidencia
            cropped_image = gondola_split[:, y_min:y_max, x_min:x_max]

            # Guardar el recorte en la carpeta correspondiente
            product_output_folder = output_folder #/ product_image_path.stem
            product_output_folder.mkdir(parents=True, exist_ok=True)
            cropped_image_np = cropped_image.cpu().permute(1, 2, 0).numpy() * 255  # Convertir a numpy
            cropped_image_np = cropped_image_np.astype(np.uint8)  # Convertir a uint8
            output_path = product_output_folder / f"{product_image_path.stem}_detected_crop.png"
            cv2.imwrite(str(output_path), cropped_image_np)

            print(f"Recorte guardado en {output_path}")

        else:
            print(f"No se encontraron suficientes coincidencias para el producto {product_image_path.name}")


Analizando el producto: EVAGINA humedas TOA x 20 - 7796285290740.jpg
Coincidencias en la mitad superior: 24, coincidencias en la mitad inferior: 7
Objeto detectado con 24 correspondencias en la partición para el producto: EVAGINA humedas TOA x 20 - 7796285290740.jpg
Recorte guardado en /Users/manu/Downloads/detecciones/EVAGINA humedas TOA x 20 - 7796285290740_detected_crop.png
Analizando el producto: EVAGINA DEO INTIMO AER x 75 - 7796285290856.jpg
Coincidencias en la mitad superior: 10, coincidencias en la mitad inferior: 8
Objeto detectado con 10 correspondencias en la partición para el producto: EVAGINA DEO INTIMO AER x 75 - 7796285290856.jpg
Recorte guardado en /Users/manu/Downloads/detecciones/EVAGINA DEO INTIMO AER x 75 - 7796285290856_detected_crop.png
Analizando el producto: EVAGINA ESP x 240 - 7796285289348.png
Coincidencias en la mitad superior: 10, coincidencias en la mitad inferior: 24
Objeto detectado con 24 correspondencias en la partición para el producto: EVAGINA ESP x 2

In [28]:
import pandas as pd

# Cargar la base de promociones
promociones_df = pd.read_csv('promociones.csv')

# Filtrar por mes 'Junio'
promociones_junio = promociones_df[promociones_df['Mes'] == 'junio']

# Lista de productos detectados en la carpeta
productos_detectados = [product_image_path.stem for product_image_path in product_folder.glob('*') 
                        if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']]
print(productos_detectados)
# Filtrar promociones por los productos en la carpeta
# Crear una nueva columna combinando 'DESCRIPCIÓN' y 'EAN NUEVO'
promociones_junio['DESCRIPCIÓN_EAN'] = promociones_junio['DESCRIPCIÓN'] + " - " + promociones_junio['EAN NUEVO']
print(promociones_junio)

# Filtrar promociones por los productos en la carpeta
promociones_filtradas = promociones_junio[promociones_junio['DESCRIPCIÓN_EAN'].isin(productos_detectados)]
promociones_filtradas


['EVAGINA humedas TOA x 20 - 7796285290740', 'EVAGINA DEO INTIMO AER x 75 - 7796285290856', 'EVAGINA ESP x 240 - 7796285289348', 'EVAGINA JLI x 200 - 7796285289324', 'EVAGINA Pomo GEL x 50 - 7796285289331']
          EAN NUEVO                               DESCRIPCIÓN Descuento   \
362   7790375268787              BAGOVIT A CLASSIC CRM X 50 G        25%   
363   7790375260941             BAGOVIT A CLASSIC CRM X 100 G        25%   
364   7790375245054             BAGOVIT A CLASSIC CRM X 200 G        25%   
365   7790375270339                 BAGOVIT A CLASSIC X 400 G        25%   
366   7790375245061               BAGOVIT A LIGHT CRM X 100 G        25%   
...             ...                                       ...        ...   
1218  7791293047812     LUX JAB GLICDO LEMON Y FLORES 108X90G        15%   
1219  7791293047874         LUX JAB GLICDO MANZANILLA 108X90G        15%   
1220  7791293050768    LUX JAB LIQ GLIC BROMELIA RFL 12X220ML        15%   
1221  7791293050775  LUX JAB LIQ 

/var/folders/q_/r_vc8qlx2gjgzy57nd_nrykw0000gn/T/ipykernel_11443/2972193121.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  promociones_junio['DESCRIPCIÓN_EAN'] = promociones_junio['DESCRIPCIÓN'] + " - " + promociones_junio['EAN NUEVO']


,EAN NUEVO,DESCRIPCIÓN,Descuento,Mes,Categoria,DESCRIPCIÓN_EAN
669,7796285289348,EVAGINA ESP x 240,40%,junio,Proteccion femenina,EVAGINA ESP x 240 - 7796285289348
670,7796285289324,EVAGINA JLI x 200,40%,junio,Proteccion femenina,EVAGINA JLI x 200 - 7796285289324
671,7796285289331,EVAGINA Pomo GEL x 50,40%,junio,Proteccion femenina,EVAGINA Pomo GEL x 50 - 7796285289331
672,7796285290856,EVAGINA DEO INTIMO AER x 75,40%,junio,Proteccion femenina,EVAGINA DEO INTIMO AER x 75 - 7796285290856


In [37]:
for idx, row in promociones_filtradas.iterrows():

    producto = row['DESCRIPCIÓN_EAN']
    descuento = row['Descuento ']  # El descuento debería estar en porcentaje, como 20, 30, etc.

    # Ruta de la imagen del descuento correspondiente
    descuento_img_path = Path(f'imagenes_descuentos/descuento_{descuento}.png')

    # Cargar la imagen del descuento
    descuento_image = load_image(descuento_img_path)

    # Realizar template matching con SuperGlue en el recorte
    recorte_producto_path = output_folder / f"{producto}_detected_crop.png"  # Suponiendo que ya guardaste los recortes
    recorte_producto_image = load_image(recorte_producto_path)

    # Convertir imágenes a blanco y negro para template matching
    descuento_image_gray = descuento_image.mean(dim=0, keepdim=True)
    recorte_producto_gray = recorte_producto_image.mean(dim=0, keepdim=True)

    # Extraer características con SuperGlue
    feats0 = extractor.extract(descuento_image_gray.to(device))
    feats1 = extractor.extract(recorte_producto_gray.to(device))

    # Coincidencias
    matches = matcher({"image0": feats0, "image1": feats1})
    feats0, feats1, matches = [rbd(x) for x in [feats0, feats1, matches]]

    # Puntos clave y coincidencias
    kpts0, kpts1, matches_found = feats0["keypoints"], feats1["keypoints"], matches["matches"]

    if len(matches_found) >= 8:
        print(f"Descuento correcto encontrado para el producto {producto} con {len(matches_found)} coincidencias.")
    else:
        print(f"Promoción incorrecta o no detectada para el producto {producto}.")


/Users/manu/Library/Python/3.9/lib/python/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Promoción incorrecta o no detectada para el producto EVAGINA ESP x 240 - 7796285289348.
Promoción incorrecta o no detectada para el producto EVAGINA JLI x 200 - 7796285289324.
Promoción incorrecta o no detectada para el producto EVAGINA Pomo GEL x 50 - 7796285289331.
Promoción incorrecta o no detectada para el producto EVAGINA DEO INTIMO AER x 75 - 7796285290856.
